## This Notebook will pull in and summarize our Pit Robot Scouter JSON Data.

The Notebook exports its results to CSV and Excel files located in a 'LatestResults' directory 
relative to this Jupyter Notebook.

** NO NEED TO RUN THE NEXT CODE SECTION IF YOU HAVE ALREADY INSTALLED THE FOLLOWING PYTHON LIBRARIES **

In [1]:
# NO NEED TO RUN THIS CODE SECTION IF YOU HAVE ALREADY INSTALLED THE FOLLOWING PYTHON LIBRARIES. 
# No problem though if you accidently run it as it will just say the "Requirement already satisfied"

!pip install requests
!pip install pandas

# The following library is currently being used for the Excel writer.  Was planning to use for exporting 
# to an Excel file format. Panda no longer supports exporting directly into an Excel file format.
!pip install openpyxl

In [2]:
import requests
import pandas as pd
from datetime import datetime

In [3]:
# Getting a unique file timestamp
currentTimeStamp = datetime.now()
file_timestamp = currentTimeStamp.strftime("%Y-%m-%d-T%H-%M")

In [4]:
# Reading in the PIT JSON file that was exported from Robot Scouter phone app
df_scouting = pd.read_json('LatestInputOfScoutData/PIT.json')

In [5]:
# Making it easier to designate the header / column name to use per metric collected.
d_labels = {'ECIgrKJfMO8I2E8Lsovh': 'Scout', 
            'ONV3DyLTXyEeLzuZZnQs': 'HARDWARE Floor Intake', 'TvyFAXJuHxPsW26qxSXm': 'HARDWARE Drivetrain', 
            'ImxowXvvNc0k4foZnkC2': 'AUTO Level', 'RdwMEdp9hqmT2nF50pn8': 'AUTO Strategy Comments',
            'zQCQI1X0XFbAdupCuZhm': 'TELEOP Where They Shoot', 'ZKt8i4uYZsEAus75zJ7o': 'TELEOP Strategy Comments',
            'tlJnSqtvEgakktTXBgkW': 'END GAME Highest Climb', 'DNmVOXCuMUKHvU6BzLMV': 'END GAME Climb Comments', 
            'LY1pu680oPokOpJNI4q3': 'DEFENSE Shown', 'MJIMd2Nj0Cf9zPEzsRYh': 'DEFENSE Comments',
            'ry5Q83c1ty8QyWcoQKsI': 'Additional Notes'
           }       

In [6]:
# Index to use for each row of pit strategy that scouts have provided
idx_allpitstrategies = 0

# A pandas DataFrame to collect all the metrics.  
# Each row is a match and we will identify the team that was scouted in the row. 
df_allpitstrategies = pd.DataFrame()

# We will collect all the metrics for the current match into a dictionary.
# This dictionary will be added to the end of the df_allmetrics dataframe.
dict_pitstrategy = {}
    
# Let's run through all the scouting match metrics data we have from our scouts
for index_team, row_team in df_scouting.iterrows():
    
    # Let's grab all the matches of metrics that we have for the current team
    df_matches = pd.DataFrame.from_dict(row_team['teams'])
    
    # We'll now iterate through each match's metrics for the current team
    for index_match, row_match in df_matches.iterrows():
        
        # Setting the team number and timestamp for the current match's metrics
        team = index_team
        timestamp = pd.Timestamp(row_match['timestamp'], unit='ms')  
        
        # Adding the team number and timestamp into the dictionary that we will add as a row of metrics
        dict_pitstrategy['Index'] = [idx_allpitstrategies]        
        dict_pitstrategy['Team'] = [team]
        dict_pitstrategy['Timestamp'] = [timestamp.strftime('%m-%d-%Y')]
        
        # When the JSON file was imported, the metrics were still in a set of key, value, category, name set of fields
        # under the 'metrics' column of the dataframe that was created when we read the JSON file.
        # We will iterate through these key, value type sets of data to parse out what we want from the scouting metrics.
        d_metrics = row_match['metrics']
        
        # Using items() + list comprehension
        # Substring Key match in dictionary
        for key, val in d_metrics.items():
            # print("key: ", key, "Type:", val['type'],"Name:", val['name'], "Category:", val['category'],"Value:", val['value'])
            # print("Label will be:", d_labels[key], 'Index:', i)

            # Adding metrics to our dictionary that will be added as a row once we have all of them for the match.
            if val['value'] is None:
                dict_pitstrategy[d_labels[key]] = ['']
            elif val['type'] in {"boolean", "number","stopwatch"}:
                dict_pitstrategy[d_labels[key]] = [val['value']]                
            else:
                dict_pitstrategy[d_labels[key]] = [val['value']]                

        
        # Let's convert our dictionary of metrics for a match into a DataFrame to be ready to add
        new_pitstrategy = pd.DataFrame(dict_pitstrategy)
        
        # If this is our first row, let's initialize our all metrics DataFrame to this first row.
        if df_allpitstrategies.empty:
            # If first row let's set it
            df_allpitstrategies = new_pitstrategy
        else:
            #append new row to the dataframe
            df_allpitstrategies = pd.concat([df_allpitstrategies, new_pitstrategy], ignore_index=True)
    
        # Incrementing to be ready to use as an index to each row
        idx_allpitstrategies = idx_allpitstrategies + 1

# Let's just see what we have!
display(df_allpitstrategies)

,Index,Team,Timestamp,Scout,HARDWARE Floor Intake,HARDWARE Drivetrain,AUTO Level,AUTO Strategy Comments,TELEOP Where They Shoot,TELEOP Strategy Comments,END GAME Highest Climb,END GAME Climb Comments,DEFENSE Shown,DEFENSE Comments,Additional Notes
0,0,1011,03-24-2022,Meh,True,Standard 6/8 wheel,Unknown,,Unknown,,Unknown,,False,,Last minute construction in pit
1,1,1245,03-24-2022,NG MEH,True,Omni/Mecanum,1-Taxi only,They got stuck on another robot during auto. (...,Upper Hub only,,Mid,Went in after the 30sec mark. Barely got off t...,False,,Slow moving
2,2,1245,03-25-2022,MH,True,Unknown,3-Shoot + Taxi only (1 Cargo),,Upper Hub only,Manual aim only,Mid,Not attempted in Q1,False,,Their intake is not efficient and could cause ...
3,3,1303,03-24-2022,NG MEH,True,Omni/Mecanum,Unknown,,Upper Hub only,,Mid,,False,,Didn't move. (MEH) Having problems passing in...
4,4,1332,03-24-2022,CM MM NG MEH,True,Standard 6/8 wheel,1-Taxi only,Not sure they went far enough. (MEH) They say ...,Lower Hub only,Shoots from right up against hub\nNG - shootin...,Mid,Surprisingly fast for a screw drive,True,(MEH) 8 motors on chassis. 90 lbs. Very fast. ...,
5,5,1339,03-17-2022,JB MEH NG,True,Swerve,5-More than 2 Cargo combination,(NG) Saw 2 ball auto. Didn't shoot 2nd ball un...,Upper Hub only,Must shoot from the base of the hub.\n\n(NG) F...,Traversal,Takes a awhile to get there(20-30 seconds). Dr...,True,Did incur penalties. Was able to bump an oppon...,(MEH) Faster
6,6,1410,03-24-2022,Meh,True,Standard 6/8 wheel,4-Shoot + Taxi + Intake (2 Cargo max),,Lower Hub only,,Traversal,35 sec,True,,
7,7,159,03-24-2022,MN,True,Standard 6/8 wheel,3-Shoot + Taxi only (1 Cargo),,Upper Hub only,,High,,False,,Cycle time to shoot was long.
8,8,159,03-24-2022,NG KG,True,Standard 6/8 wheel,1-Taxi only,Didn't quite leave tarmac,Upper Hub only,"Attempts take long time, don't stay in\nBall f...",Mid,"Fast climb, dropped down, then went back up",False,Didn't this match,KG_Stopped part way( battery fell out)
9,9,1619,03-16-2022,JB,True,Swerve,5-More than 2 Cargo combination,Setup on the opposite side as the hangar in th...,Upper Hub only,They like the launchpad when heavily defended....,Traversal,15 sec climb. They go into the Hangar with 30 ...,False,Never seen them play defense.,


In [7]:
# Let's see how many matches and what types of columns we have.
df_allpitstrategies.info()
df_allpitstrategies.drop('Index',axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Index                     60 non-null     int64 
 1   Team                      60 non-null     int64 
 2   Timestamp                 60 non-null     object
 3   Scout                     60 non-null     object
 4   HARDWARE Floor Intake     60 non-null     bool  
 5   HARDWARE Drivetrain       60 non-null     object
 6   AUTO Level                60 non-null     object
 7   AUTO Strategy Comments    60 non-null     object
 8   TELEOP Where They Shoot   60 non-null     object
 9   TELEOP Strategy Comments  60 non-null     object
 10  END GAME Highest Climb    60 non-null     object
 11  END GAME Climb Comments   60 non-null     object
 12  DEFENSE Shown             60 non-null     bool  
 13  DEFENSE Comments          60 non-null     object
 14  Additional Notes          60

,Team,Timestamp,Scout,HARDWARE Floor Intake,HARDWARE Drivetrain,AUTO Level,AUTO Strategy Comments,TELEOP Where They Shoot,TELEOP Strategy Comments,END GAME Highest Climb,END GAME Climb Comments,DEFENSE Shown,DEFENSE Comments,Additional Notes
0,1011,03-24-2022,Meh,True,Standard 6/8 wheel,Unknown,,Unknown,,Unknown,,False,,Last minute construction in pit
1,1245,03-24-2022,NG MEH,True,Omni/Mecanum,1-Taxi only,They got stuck on another robot during auto. (...,Upper Hub only,,Mid,Went in after the 30sec mark. Barely got off t...,False,,Slow moving
2,1245,03-25-2022,MH,True,Unknown,3-Shoot + Taxi only (1 Cargo),,Upper Hub only,Manual aim only,Mid,Not attempted in Q1,False,,Their intake is not efficient and could cause ...
3,1303,03-24-2022,NG MEH,True,Omni/Mecanum,Unknown,,Upper Hub only,,Mid,,False,,Didn't move. (MEH) Having problems passing in...
4,1332,03-24-2022,CM MM NG MEH,True,Standard 6/8 wheel,1-Taxi only,Not sure they went far enough. (MEH) They say ...,Lower Hub only,Shoots from right up against hub\nNG - shootin...,Mid,Surprisingly fast for a screw drive,True,(MEH) 8 motors on chassis. 90 lbs. Very fast. ...,
5,1339,03-17-2022,JB MEH NG,True,Swerve,5-More than 2 Cargo combination,(NG) Saw 2 ball auto. Didn't shoot 2nd ball un...,Upper Hub only,Must shoot from the base of the hub.\n\n(NG) F...,Traversal,Takes a awhile to get there(20-30 seconds). Dr...,True,Did incur penalties. Was able to bump an oppon...,(MEH) Faster
6,1410,03-24-2022,Meh,True,Standard 6/8 wheel,4-Shoot + Taxi + Intake (2 Cargo max),,Lower Hub only,,Traversal,35 sec,True,,
7,159,03-24-2022,MN,True,Standard 6/8 wheel,3-Shoot + Taxi only (1 Cargo),,Upper Hub only,,High,,False,,Cycle time to shoot was long.
8,159,03-24-2022,NG KG,True,Standard 6/8 wheel,1-Taxi only,Didn't quite leave tarmac,Upper Hub only,"Attempts take long time, don't stay in\nBall f...",Mid,"Fast climb, dropped down, then went back up",False,Didn't this match,KG_Stopped part way( battery fell out)
9,1619,03-16-2022,JB,True,Swerve,5-More than 2 Cargo combination,Setup on the opposite side as the hangar in th...,Upper Hub only,They like the launchpad when heavily defended....,Traversal,15 sec climb. They go into the Hangar with 30 ...,False,Never seen them play defense.,


In [8]:
# Let's transpose our data to have all the Pit fields along the left side and the teams in a column
dfT_allpitstrategies = df_allpitstrategies.T

## Output the full list of Pit submissions we have from the Scouts

In [9]:
#  All match metrics exporting to a CSV file to be used by other scripts or to look at AS-IS
df_allpitstrategies[['Team','Timestamp','Scout','HARDWARE Floor Intake','HARDWARE Drivetrain','AUTO Level','AUTO Strategy Comments','TELEOP Where They Shoot','TELEOP Strategy Comments','END GAME Highest Climb','END GAME Climb Comments','DEFENSE Shown','DEFENSE Comments','Additional Notes']].to_csv("LatestResults/AllTeamStrategies.csv", index=False, float_format="%.1f")

In [10]:
#  All match metrics exporting to a CSV file to be used by other scripts or to look at AS-IS
dfT_allpitstrategies.to_csv("LatestResults/AllTeamStrategies_FieldsOnLeft.csv", index=False, float_format="%.1f")

In [11]:
# Attempting to write to Excel to look at AS-IS 
filename_Excel = "LatestResults/OurScouting.xlsx"
with pd.ExcelWriter(filename_Excel, mode='a', if_sheet_exists='overlay', datetime_format='mmm d yyyy hh:mm:ss', date_format='hh:mm:ss.000', engine="openpyxl") as writer:
    dfT_allpitstrategies.to_excel(writer, sheet_name="Team Strategies-inColumns", index=True, float_format="%.1f")
    df_allpitstrategies[['Team','Timestamp','Scout','HARDWARE Floor Intake','HARDWARE Drivetrain','AUTO Level','AUTO Strategy Comments','TELEOP Where They Shoot','TELEOP Strategy Comments','END GAME Highest Climb','END GAME Climb Comments','DEFENSE Shown','DEFENSE Comments','Additional Notes']].to_excel(writer, sheet_name="Team Strategies-inRows", index=False, float_format="%.1f")


In [12]:
# Let's get our team's Qualification Match schedule that was outputted as a CSV file that we can read
df_ourQMSchedule = pd.read_csv("LatestInputOfScoutData/Colorado 2022 Component OPR Calculator - Team Schedule.csv")

In [13]:
# Writing to Excel all the Pit Strategies that we have for each team in a Qualification Match we are in
# Each Sheet in the Excel Workbook will be a Qualification Match
filename_Excel = "LatestResults/OurScouting-QMs.xlsx"
with pd.ExcelWriter(filename_Excel, datetime_format='mmm d yyyy hh:mm:ss', date_format='hh:mm:ss.000') as writer:

    for index, qm in df_ourQMSchedule.iterrows():
    
        df_qm = pd.DataFrame()
    
        # Grab PIT info for Red Team #1
        df_qm = df_allpitstrategies.loc[df_allpitstrategies['Team'] == qm['Red 1']] 
    
        # Grab PIT info for Red Team #2
        df_qm = pd.concat([df_qm, df_allpitstrategies.loc[df_allpitstrategies['Team'] == qm['Red 2']] ])

        # Grab PIT info for Red Team #3
        df_qm = pd.concat([df_qm, df_allpitstrategies.loc[df_allpitstrategies['Team'] == qm['Red 3']] ])

        # Grab PIT info for Blue Team #1
        df_qm = pd.concat([df_qm, df_allpitstrategies.loc[df_allpitstrategies['Team'] == qm['Blue 1']] ])
    
        # Grab PIT info for Blue Team #2
        df_qm = pd.concat([df_qm, df_allpitstrategies.loc[df_allpitstrategies['Team'] == qm['Blue 2']] ])

        # Grab PIT info for Blue Team #3
        df_qm = pd.concat([df_qm, df_allpitstrategies.loc[df_allpitstrategies['Team'] == qm['Blue 3']] ])

        df_qm.drop('Index',axis=1)
        dfT_qm = df_qm.T

        dfT_qm.to_excel(writer, sheet_name=qm['Match'], index=True, float_format="%.1f")
 